In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip list

Package               Version
--------------------- -----------
altair                4.2.0
argon2-cffi           21.3.0
argon2-cffi-bindings  21.2.0
asttokens             2.0.8
attrs                 22.1.0
autoviz               0.1.43
backcall              0.2.0
beautifulsoup4        4.11.1
bleach                5.0.1
bokeh                 2.4.3
certifi               2022.6.15
cffi                  1.15.1
charset-normalizer    2.1.1
click                 8.1.3
colorama              0.4.5
colorcet              3.0.0
cycler                0.11.0
dataclasses           0.6
debugpy               1.6.3
decorator             5.1.1
defusedxml            0.7.1
emoji                 2.0.0
entrypoints           0.4
evidently             0.1.56.dev0
executing             0.10.0
fastjsonschema        2.16.1
fonttools             4.37.0
fsspec                2022.7.1
greenlet              1.1.3
holoviews             1.14.9
htmlmin               0.1.12
hvplot                0.8.0
idna               


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Libraries

In [3]:
# import sys
# print(sys.path)
# sys.path.insert(0, "C:\\Users\\KonuTech\\gfk_data_analyst\\gfk_ml_ops_data_analyst\\scripts\\python\\")

In [4]:
import os
# import sys
import json
import numpy as np
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
# import math
from pandas_profiling import ProfileReport
# import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scripts.python.get_metrics import *
from scripts.python.get_charts import get_monthly_stability_chart

In [5]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [6]:
INPUT_PATH="input"
OUTPUT_PATH="output"
CONFIG_PATH="config"

INPUT_DATA="test_data2"
INPUT_DATA_CONFIG="input_config"
OUTPUT_DATA="output"

INPUT_EXTENSION="csv"
INPUT_CONFIG_EXTENSION="json"
OUTPUT_EXTENSION="csv"

INPUT_FILE=f"{INPUT_DATA}.{INPUT_EXTENSION}"
INPUT_CONFIG_FILE=f"{INPUT_DATA_CONFIG}.{INPUT_CONFIG_EXTENSION}"
OUTPUT_FILE=f"{OUTPUT_DATA}.{OUTPUT_EXTENSION}"

INPUT_ABS_APTH=os.path.abspath(os.path.join(INPUT_PATH, INPUT_FILE))
INPUT_FILE_CONFIG=os.path.abspath(os.path.join(CONFIG_PATH, INPUT_CONFIG_FILE))
OUTPUT_ABS_APTH=os.path.abspath(os.path.join(OUTPUT_PATH, OUTPUT_FILE))

In [7]:
with open(INPUT_FILE_CONFIG, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [8]:
CONFIG

{'INPUTS': {'FILE_NAME': ['test_data2.csv'],
  'SEPARATOR': ',',
  'DECIMAL': None,
  'ENCODING': 'utf-8',
  'FLOAT_PRECISION': 'high',
  'INDEXES': ['period_end_date', 'translated_when'],
  'DATE_COLUMNS': ['period_end_date', 'translated_when'],
  'DTYPE': {'if_data_corrected': 'object',
   'prod_gr_id': 'object',
   'country_id_n': 'object',
   'delivery_type_id': 'object',
   'freq_id': 'object',
   'retailer_id': 'object',
   'brand_id': 'object',
   'predict_automatch': 'float',
   'class_acctual': 'float'},
  'COLUMNS_WITH_NAN_VALUES': [None]},
 'MODEL': [None],
 'OUTPUTS': {'NAME': ['output.csv']}}

# Load data

In [9]:
# LOAD CSV
df = pd.read_csv(
    INPUT_ABS_APTH,
#     index_col=CONFIG['INPUTS']['INDEXES'],
    sep=CONFIG['INPUTS']['SEPARATOR'],
    # quotechar='"',
    # thousands=',',
#     decimal=CONFIG['INPUTS']['DECIMAL'],
    encoding=CONFIG['INPUTS']['ENCODING'],
    infer_datetime_format=True,
    parse_dates=CONFIG['INPUTS']['DATE_COLUMNS'],
    engine="c",
    low_memory=False,
#     float_precision=CONFIG['INPUTS']['DATE_COLUMNS'],
    skipinitialspace=True,
    dtype=CONFIG['INPUTS']['DTYPE']
)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19697 entries, 0 to 19696
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   period_end_date    19640 non-null  datetime64[ns, UTC]
 1   translated_when    19697 non-null  datetime64[ns, UTC]
 2   if_data_corrected  19697 non-null  object             
 3   prod_gr_id         19697 non-null  object             
 4   country_id_n       18405 non-null  object             
 5   delivery_type_id   18362 non-null  object             
 6   freq_id            19697 non-null  object             
 7   retailer_id        19697 non-null  object             
 8   brand_id           19697 non-null  object             
 9   predict_automatch  19368 non-null  float64            
 10  class_acctual      19697 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(2), object(7)
memory usage: 1.7+ MB


In [11]:
df.sort_values(by=['translated_when'], inplace=True)

In [12]:
df.head(10)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
1908,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,0,426,126,22866,1,174,280,1.0,1.0
1907,2020-08-30 00:00:00+00:00,2020-09-01 03:35:26+00:00,0,426,126,22866,2,93,96,1.0,1.0
4503,2020-08-30 00:00:00+00:00,2020-09-01 03:44:29+00:00,0,413,113,12731,2,95,182,1.0,1.0
8009,2020-09-01 00:00:00+00:00,2020-09-01 06:15:32+00:00,1,426,114,18614,1,173,279,1.0,1.0
5091,2020-09-01 00:00:00+00:00,2020-09-01 06:15:50+00:00,1,426,114,18614,1,34,234,1.0,1.0
8010,2020-09-01 00:00:00+00:00,2020-09-01 06:21:31+00:00,1,426,114,18614,2,93,96,1.0,1.0
9945,2020-09-01 00:00:00+00:00,2020-09-01 06:24:04+00:00,1,426,114,18614,1,117,243,1.0,1.0
9944,2020-09-01 00:00:00+00:00,2020-09-01 06:24:13+00:00,1,426,114,18614,2,11,33,1.0,1.0
10999,2020-09-01 00:00:00+00:00,2020-09-01 06:25:53+00:00,1,426,114,18614,1,26,169,0.0,0.0
9947,2020-09-01 00:00:00+00:00,2020-09-01 06:26:45+00:00,1,426,114,18614,1,33,379,0.0,0.0


In [13]:
df.describe(
    include='all',
    datetime_is_numeric=True
)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
count,19640,19697,19697,19697,18405,18362,19697,19697,19697,19368.000000,19697.000000
unique,NaN,NaN,2,3,34,914,2,52,199,NaN,NaN
top,NaN,NaN,0,426,121,31480,2,30,33,NaN,NaN
freq,NaN,NaN,17085,11844,4153,998,11934,1197,587,NaN,NaN
mean,2020-10-14 23:30:22.729124096+00:00,2020-10-22 06:01:21.525206784+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.636307,0.743210
min,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,2020-09-27 00:00:00+00:00,2020-09-29 13:45:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,2020-10-18 00:00:00+00:00,2020-10-22 04:30:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
75%,2020-11-08 00:00:00+00:00,2020-11-13 11:57:53+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
max,2020-12-01 00:00:00+00:00,2021-02-01 14:50:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000


# Pandas data profiler

In [14]:
profile = ProfileReport(
    df,
    title="Pandas Profiling Report",
    pool_size=0,
    infer_dtypes=False
)

In [15]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Offline example of Pandas Profiler widget:

![image info](./docs/images/pandas_profiler/profiler_example.png)

In [16]:
for col in df.columns:
    if col not in [
        'period_end_date',
        'translated_when',
        'month_year',
        'predict_automatch',
#         'country_id_n',
#         'delivery_type_id',
#         'retailer_id',
#         'brand_id',
        'class_acctual',
#         'freq_id',
#         'if_data_corrected'
    ]:
        print(col)
        get_monthly_stability_chart(
            col=col,
            data_frame=df,
            date_column="period_end_date",
            column_to_group_by=col,
            column_to_count='predict_automatch'
        )

if_data_corrected
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  if_data_corrected  VARIABLE CLASS:  0
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  if_data_corrected  VARIABLE CLASS:  1
prod_gr_id
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  prod_gr_id  VARIABLE CLASS:  426
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  prod_gr_id  VARIABLE CLASS:  427
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  prod_gr_id  VARIABLE CLASS:  413
country_id_n
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  1010
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  1002
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  150
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  103
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  country_id_n  VARIABL

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  39113
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  47850
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  27210
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  52863
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  65403
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  72827
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  57174
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  22867
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  18257
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_ty

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  37589
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71310
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  52537
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71843
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  7932
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  22146
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  75253
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  36120
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  68429
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_typ

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  42930
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  29997
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  23944
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  56462
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  13361
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  7270
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  75078
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  42858
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  69228
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_typ

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  47421
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  64967
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  19781
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  72349
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  67349
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  73658
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  40036
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  25682
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  78033
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_ty

...

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  41133
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  34038
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  70350
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71021
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  21942
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  60425
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  33353
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  65560
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71271
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_ty

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  57153
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  64965
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  55440
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  40340
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71604
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  77826
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  19105
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  36845
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  62626
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  delivery_ty

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  91
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  116
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  119
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  120
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  127
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  175
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  22
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  29
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  118
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  retailer_id  VARIABLE CLASS:  134
PRODUCED A CHART OF predict_automatch MONTH

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  204
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  286
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  213
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  200
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  243
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  35
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  247
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  248
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  127
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  58
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  

PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  208
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  132
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  167
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  253
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  427
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  49
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  20
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  241
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  382
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  VARIABLE CLASS:  379
PRODUCED A CHART OF predict_automatch MONTHLY STABILITY FOR:  brand_id  

In [ ]:
for col in df.columns:
    if col not in [
        'period_end_date',
        'translated_when',
        'month_year',
        'class_acctual',
#         'country_id_n',
#         'delivery_type_id',
#         'retailer_id',
#         'brand_id',
        'predict_automatch',
#         'freq_id',
#         'if_data_corrected'
    ]:
        print(col)
        get_monthly_stability_chart(
            col=col,
            data_frame=df,
            date_column="period_end_date",
            column_to_group_by=col,
            column_to_count='class_acctual'
        )

if_data_corrected
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  if_data_corrected  VARIABLE CLASS:  0
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  if_data_corrected  VARIABLE CLASS:  1
prod_gr_id
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  prod_gr_id  VARIABLE CLASS:  426
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  prod_gr_id  VARIABLE CLASS:  427
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  prod_gr_id  VARIABLE CLASS:  413
country_id_n
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  1010
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  1002
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  150
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  103
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  country_id_n  VARIABLE CLASS:  177
PRODUCED A CHART OF class_

PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  57174
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  22867
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  18257
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  49246
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  65132
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  76083
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  77579
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  77302
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  59646
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  35633
PRODUCED A

PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  75253
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  36120
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  68429
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  61260
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  70312
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  46412
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  6973
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  70930
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  67781
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  73968
PRODUCED A 

PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  17683
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  29052
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  62206
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  64781
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  73516
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  22991
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  64063
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  31571
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  37947
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  62228
PRODUCED A

PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  23144
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  62529
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  49248
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  75591
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  72882
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71613
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  63816
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  77994
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  64352
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  72570
PRODUCED A

PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  75844
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  66226
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  71040
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  15656
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  56040
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  59553
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  53405
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  12380
PRODUCED A CHART OF class_acctual MONTHLY STABILITY FOR:  delivery_type_id  VARIABLE CLASS:  64552


**An example of model's underperformance:**

country_id == '106'

./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg

Observed:

![image info](./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg)


./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg

Predicted:

![image info](./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg)

In [ ]:
df_sub = df[['period_end_date', 'translated_when', 'country_id_n', 'prod_gr_id', 'predict_automatch', 'class_acctual']]

In [ ]:
pysqldf(
    """
    SELECT
        'PREDICTED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        ,country_id_n
        ,predict_automatch
        --,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        ,country_id_n
        ,predict_automatch
        --,class_acctual
    ;
    """
)

In [ ]:
pysqldf(
    """
    SELECT
        'OBSERVED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        ,country_id_n
        --,predict_automatch
        ,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        ,country_id_n
        --,predict_automatch
        ,class_acctual
    ;
    """
)

In [ ]:
pysqldf(
    """
    SELECT
        'OBSERVED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ;
    """
)

In [ ]:
pysqldf(
    """
    SELECT
        'PREDICTED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ;
    """
)

In [ ]:
pysqldf(
    """
    SELECT
        'OBSERVED' AS type,
        strftime('%Y-%m', translated_when) AS month_year_translated
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year_translated
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ;
    """
)

# Something fishy

# Demand or Supplie dropped ? A product got out of fashion?

In [ ]:
pysqldf(
    """
    SELECT DISTINCT
        prod_gr_id
        ,COUNT(prod_gr_id)
    FROM (
    SELECT
        strftime('%Y-%m', period_end_date) AS month_year
        ,translated_when
        ,prod_gr_id
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        strftime('%Y', translated_when) < '2021'
        --country_id_n = '105'
        --AND month_year = '2020-11'
    --GROUP BY
        --month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    )
    GROUP BY
    prod_gr_id
    ;
    """
)

In [ ]:
pysqldf(
    """
    SELECT DISTINCT
        prod_gr_id
        ,COUNT(prod_gr_id)
    FROM (
    SELECT
        strftime('%Y-%m', period_end_date) AS month_year
        ,translated_when
        ,prod_gr_id
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        strftime('%Y', translated_when) >= '2021'
        --country_id_n = '105'
        --AND month_year = '2020-11'
    --GROUP BY
        --month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    )
    GROUP BY
    prod_gr_id
    ;
    """
)

In [ ]:
# pysqldf(
#     """
#     SELECT
#         strftime('%Y-%m', period_end_date) AS month_year
#         ,country_id_n
#         ,predict_automatch
#         ,class_acctual
#         --,SUM(predict_automatch) AS predict_automatch_sum
#         --,SUM(class_acctual) AS class_acctual_sum
#     FROM df_sub
#     WHERE
#         country_id_n = '105'
#         AND month_year = '2020-11'
#     --GROUP BY
#         --month_year
#         --,country_id_n
#         --,predict_automatch
#         --,class_acctual
#     ;
#     """
# )

# Post-Processing Bias Metrics for the Trained Model

## Difference in positive proportion in predicted labels (DPPL)

![image info](./docs/images/metrics/DPPL.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)


### CHARACTERISTICS OF METRIC:
> DPPPL > -1 AND DPPPL < 1

> For example, if the model grants loans to 50% of class 2 and to 60% of class 1, then it may be biased against class 2.
We would have to decide whether a 10% difference is material.

## VARIABLE: prod_gr_id

### prod_gr_id total count

In [ ]:
total_counts = df.groupby(['prod_gr_id']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

total_counts

### prod_gr_id by predicted classes

In [ ]:
grouped_counts = df.groupby(['prod_gr_id', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

# grouped_counts

In [ ]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == 1]

# positive_grouped_counts

In [ ]:
merged_counts = pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

# merged_counts

### (DPPL) for prod_gr_id

In [ ]:
ppl_prod_gr_id = get_PPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

ppl_prod_gr_id

In [ ]:
dppl_prod_gr_id = get_DPPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

dppl_prod_gr_id

#### Bucketizing dppl_prod_gr_id

In [ ]:
print(pd.cut(dppl_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> Basing solely on **DPPL** we can loosely assume that there is no disparity impact among **prod_gr_id** Classes

## VARIABLE: country_id_n

### country_id_n total count

In [ ]:
total_counts = df.groupby(['country_id_n']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# total_counts

### country_id_n by predicted classes

In [ ]:
grouped_counts = df.groupby(['country_id_n', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# grouped_counts

In [ ]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == 1]

# positive_grouped_counts

In [ ]:
merged_counts = pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

# merged_counts

### (DPPL) for country_id_n

In [ ]:
ppl_country_id_n = get_PPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

ppl_country_id_n

In [ ]:
dppl_country_id_n = get_DPPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

dppl_country_id_n

#### Bucketizing dppl_country_id_n

In [ ]:
print(pd.cut(dppl_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> Although it is likely. Basing solely on **DPPL** we can't assume if there is a disparity impact among **prod_gr_id** Classes yet. 

## Disparate (Adverse) Impact (DI)

![image info](./docs/images/metrics/DI.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

### CHARACTERISTICS OF METRIC:
> **DI >= 0**

> This measure may be considered fair if it resides in the **<0.8; 1.2>** range.

## (DI) for Variable: prod_gr_id

In [ ]:
ppl_prod_gr_id

In [ ]:
di_prod_gr_id = sorted([abs(i / j) for i in ppl_prod_gr_id for j in ppl_prod_gr_id if i != j])

#### Bucketizing di_prod_gr_id

In [ ]:
print(pd.cut(di_prod_gr_id, bins=4).value_counts())

#### CONCLUSIONS:

> For each possible relation between Classes the metric resides between **<0.8; 1.2>**

> We can loosely assume that there is no disparity impact between Classes of **prod_gr_id** Variable

## (DI) for Variable: country_id_n

In [ ]:
ppl_country_id_n

In [ ]:
di_country_id_n = sorted([abs(i / j) for i in ppl_country_id_n for j in ppl_country_id_n if i != j])

#### Bucketizing di_country_id_n

In [ ]:
print(pd.cut(di_country_id_n, bins=4).value_counts())

> Dividing Ratios by Ratios, so values close to 1 are good

In [ ]:
pd.Series(di_country_id_n).describe()

#### CONCLUSIONS:

> There is more than 100 Classes of Variable **country_id_n** out of **<0.8; 1.2>** bounds.

> At this moment we can loosely assume that there are some examples of Disparity Impact in case of **country_id_n** Variable.

> Seems like **DI** tries to tell that distributions between countrie **106** and **1011** are diffrent from each other. If that is true I would recommend reducing dimmensionality by feature engineering. I would use **WoE** (**Weight of Evidence**) to do so.

## Difference in Conditional Outcome (DCO)
> Type 1: **Difference in Conditional Acceptance (DCA)**

> Type 2: **Difference in Conditional Rejection (DCR)**

> When both **DCA** and **DCR** are very close to **0**, we can conclude that the proportion of qualified (as suggested by observed labels) applicants accepted by the model and the proportion of unqualified applicants rejected are nearly equal across both classes.

### Difference in Conditional Acceptance (DCA)

![image info](./docs/images/metrics/DCA.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded

> Zero denominator is possible. In such case the allocations to each Class are too small and a warning should be issued.

## (DCA) for Variable: prod_gr_id

In [ ]:
ca_prod_gr_id = get_CA(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

ca_prod_gr_id

In [ ]:
dca_prod_gr_id = get_DCA(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dca_prod_gr_id

#### Bucketizing dca_prod_gr_id

In [ ]:
print(pd.cut(dca_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO: Diffrances are really small

## (DCA) for Variable: country_id_n

In [ ]:
ca_country_id_n = get_CA(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

ca_country_id_n

In [ ]:
df[df['country_id_n']=='109'].describe()

> from the CA and mean we can assume that the Model underperforms when 'country_id_n'=='109'

In [ ]:
dca_country_id_n = get_DCA(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dca_country_id_n

#### Bucketizing dca_country_id_n

In [ ]:
print(pd.cut(dca_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

### Difference in Conditional Rejection (DCR)

![image info](./docs/images/metrics/DCR.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded

> Zero denominator is possible. In such case the allocations to each Class are too small and a warning should be issued.

## (DCR) for Variable: prod_gr_id

In [ ]:
cr_prod_gr_id = get_CR(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

cr_prod_gr_id

In [ ]:
dcr_prod_gr_id = get_DCR(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dcr_prod_gr_id

#### Bucketizing dcr_prod_gr_id

In [ ]:
print(pd.cut(dcr_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (DCR) for Variable: country_id_n

In [ ]:
cr_country_id_n = get_CR(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

cr_country_id_n

In [ ]:
dcr_country_id_n = get_DCR(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dcr_country_id_n

#### Bucketizing dcr_country_id_n

In [ ]:
print(pd.cut(dcr_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

In [ ]:
cm_by_class_prod_gr_id = get_class_cm(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

cm_by_class_prod_gr_id

In [ ]:
rd_prod_gr_id = get_RD(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rd_prod_gr_id

#### Bucketizing rd_prod_gr_id

In [ ]:
print(pd.cut(rd_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (RD) for Variable: country_id_n

In [ ]:
cm_by_class_prod_gr_id = get_class_cm(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

cm_by_class_prod_gr_id

In [ ]:
recall_by_class = get_RD(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

recall_by_class

#### Bucketizing recall_by_class

In [ ]:
print(pd.cut(recall_by_class, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## Difference in label rates (DLR)

![image info](./docs/images/metrics/DAR.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> DAR > -1 AND DAR < 1

> DRR > -1 AND DRR < 1

> DAR is the same as precision difference between the first and second classes.

## (DAR) for Variable: prod_gr_id

In [ ]:
ar_by_class_prod_gr_id = get_AR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

ar_by_class_prod_gr_id

In [ ]:
dar_by_class_prod_gr_id = get_DAR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

dar_by_class_prod_gr_id

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(dar_by_class_prod_gr_id, bins=3).value_counts())

## (DRR) for Variable: prod_gr_id

In [ ]:
rr_by_class_prod_gr_id = get_RR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rr_by_class_prod_gr_id

In [ ]:
drr_by_class_prod_gr_id = get_DRR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

drr_by_class_prod_gr_id

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(drr_by_class_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (DAR) for Variable: country_id_n

In [ ]:
ar_by_class_country_id_n = get_AR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

ar_by_class_country_id_n

In [ ]:
dar_by_class_country_id_n = get_DAR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

dar_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(dar_by_class_country_id_n, bins=3).value_counts())

## (DRR) for Variable: country_id_n

In [ ]:
rr_by_class_country_id_n = get_RR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

rr_by_class_country_id_n

In [ ]:
drr_by_class_country_id_n = get_DRR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

drr_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(drr_by_class_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## Accuracy DIfference (AD)

![image info](./docs/images/metrics/AD.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded metric

> Care needed when FN = 0

## (AD) for Variable: prod_gr_id

In [ ]:
a_by_class_prod_gr_id = get_A(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rr_by_class_prod_gr_id

In [ ]:
ad_by_class_country_id_n = get_AD(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

ad_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(ad_by_class_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (AD) for Variable: country_id_n

In [ ]:
a_by_class_country_id_n = get_A(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

rr_by_class_country_id_n

In [ ]:
ad_by_class_country_id_n = get_AD(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

ad_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(ad_by_class_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO: